In [2]:
import numpy as np
import json
from scipy.sparse import load_npz,save_npz,diags,csr_matrix
import scipy.sparse as sp
import pandas as pd
import os
from io import BytesIO
from tqdm import tqdm
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.backends.backend_pdf import PdfPages
from pypdf import PdfReader, PdfWriter
from tempfile import NamedTemporaryFile
import networkx as nx
import pickle
import gseapy as gp
import mygene
from IPython.display import display, HTML

In [3]:
pd.set_option('display.width', None)      # No line-wrapping
pd.set_option('display.max_columns', None)  # Show all columns

In [4]:
DISEASE = "BIPOLAR"
RESULT_FOLDER = f"../output/{DISEASE}/leiden_results"
DGIDB_DIRECTORY = f"../../Gen_Hypergraph/output/DGIDB_{DISEASE}/"
MSIGDB_DIRECTORY = "../../Gen_Hypergraph/output/MSigDB_Full/"

with open(DGIDB_DIRECTORY + "gene_to_index.json", "r") as file:
    dgidb = json.load(file)
with open(MSIGDB_DIRECTORY + "gene_to_index.json", "r") as file:
    msigdb = json.load(file)

In [5]:
def communities_cutoff(communities, cutoff = 100):
    result = []
    for community in communities:
        if len(community) >= cutoff:
            result.append(community)

    return result, len(result)

def community_central_genes_by_num(G, community_nodes, weight="weight", top_n=20):
    C = set(community_nodes)
    H = G.subgraph(C).copy()                       # induced subgraph
    # within-community (weighted) degree
    k = {u: H.degree(u, weight=weight) for u in H}
    ks = np.array(list(k.values()), dtype=float)
    mu, sigma = ks.mean(), ks.std() if ks.std() > 0 else 1.0
    Z = {u: (k[u] - mu)/sigma for u in H}          # within-module degree z-score

    # rank by z
    ranked = sorted(H.nodes(), key=lambda u: (Z[u]), reverse=True)
    return [u for u in ranked[:top_n]]

def community_central_genes_by_score(G, community_nodes, weight="weight",score_cap = 1):
    C = set(community_nodes)
    H = G.subgraph(C).copy()                       # induced subgraph
    # within-community (weighted) degree
    k = {u: H.degree(u, weight=weight) for u in H}
    ks = np.array(list(k.values()), dtype=float)
    mu, sigma = ks.mean(), ks.std() if ks.std() > 0 else 1.0
    Z = {u: (k[u] - mu)/sigma for u in H}          # within-module degree z-score

    # rank by z
    ranked = sorted(H.nodes(), key=lambda u: (Z[u]), reverse=True)
    return [u for u in ranked if Z[u] >= score_cap]

def community_central_genes_by_pct(G, community_nodes, weight="weight",pct = 0.3):
    C = set(community_nodes)
    H = G.subgraph(C).copy()                       # induced subgraph
    # within-community (weighted) degree
    k = {u: H.degree(u, weight=weight) for u in H}
    ks = np.array(list(k.values()), dtype=float)
    mu, sigma = ks.mean(), ks.std() if ks.std() > 0 else 1.0
    Z = {u: (k[u] - mu)/sigma for u in H}          # within-module degree z-score

    # rank by z
    ranked = sorted(H.nodes(), key=lambda u: (Z[u]), reverse=True)
    top = int(len(ranked)*pct)
    return [u for u in ranked[:top]]

In [6]:
# Combine indices from both layers, first DGIDB and then MSIGDB
dgidb_rev = {index : gene for gene,index in dgidb.items()}
msigdb_rev = {index : gene for gene,index in msigdb.items()}

a_max = max(dgidb_rev.keys())
b_min = min(msigdb_rev.keys())
shift = (a_max + 1) - b_min   # ensures no overlap
index_to_gene = {**dgidb_rev, **{k + shift: v for k, v in msigdb_rev.items()}}

print(dgidb_rev)
print(msigdb_rev)
print(index_to_gene)

{0: '1565', 1: '5468', 2: '79915', 3: '5999', 4: '5594', 5: '185', 6: '3791', 7: '142', 8: '318', 9: '1838', 10: '2159', 11: '6263', 12: '3815', 13: '6351', 14: '1437', 15: '29994', 16: '3725', 17: '10919', 18: '283106', 19: '153', 20: '6336', 21: '358', 22: '2261', 23: '155', 24: '5340', 25: '3752', 26: '6158', 27: '2099', 28: '51426', 29: '9290', 30: '3091', 31: '1813', 32: '303', 33: '3276', 34: '2260', 35: '9734', 36: '2908', 37: '10966', 38: '776', 39: '9971', 40: '4088', 41: '1436', 42: '817', 43: '3570', 44: '6198', 45: '147746', 46: '7468', 47: '1559', 48: '5742', 49: '360', 50: '552', 51: '3156', 52: '9682', 53: '29126', 54: '2651', 55: '10298', 56: '2064', 57: '427', 58: '2906', 59: '1906', 60: '354', 61: '3952', 62: '2648', 63: '5893', 64: '5467', 65: '1395', 66: '2568', 67: '926', 68: '5605', 69: '2357', 70: '1080', 71: '2185', 72: '256815', 73: '5604', 74: '2740', 75: '146850', 76: '5916', 77: '695', 78: '3480', 79: '4193', 80: '7015', 81: '1956', 82: '238', 83: '2237', 84

In [7]:
# Sanity Check; Results should be equal
print(index_to_gene[12678+len(dgidb_rev)],
msigdb_rev[12678])
print(index_to_gene[26754])

65996 65996
129810495


In [8]:
# Loading result graph and communities
with open(f"{RESULT_FOLDER}/result_communities.pkl", "rb") as f:
    communities = pickle.load(f)
with open(f"{RESULT_FOLDER}/result_graph.pkl", "rb") as f:
    graph = pickle.load(f)

In [9]:
# Checking communities
communities_greater_100, num_greater_100 = communities_cutoff(communities,100)
print(communities_greater_100, '\n', num_greater_100)
print(len(communities))

[[1, 8, 10, 13, 16, 18, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 38, 39, 43, 45, 49, 57, 77, 87, 88, 89, 90, 91, 92, 100, 113, 115, 116, 117, 119, 125, 127, 130, 131, 137, 139, 141, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 161, 162, 163, 164, 167, 169, 170, 171, 172, 174, 176, 177, 185, 186, 196, 204, 209, 210, 220, 222, 224, 225, 226, 235, 237, 243, 245, 246, 247, 248, 253, 254, 255, 260, 266, 271, 272, 274, 285, 295, 296, 302, 304, 305, 320, 325, 326, 327, 335, 337, 344, 346, 348, 350, 353, 356, 358, 363, 375, 376, 378, 383, 384, 385, 387, 392, 393, 402, 405, 408, 417, 419, 420, 432, 437, 440, 446, 455, 456, 457, 459, 470, 489, 494, 506, 515, 518, 519, 535, 536, 545, 547, 549, 556, 560, 561, 563, 565, 566, 567, 569, 571, 585, 586, 587, 589, 590, 594, 595, 600, 602, 610, 615, 616, 621, 626, 631, 633, 634, 635, 637, 647, 653, 654, 658, 692, 695, 707, 711, 718, 721, 724, 748, 766, 769, 780, 789, 793, 795, 798, 802, 803, 808, 812, 820, 821, 822, 823, 828, 832, 833

In [10]:
# Update communities to include only the important genes, determined by a community-size_cap and z-score_cap
size_cap = 50
pct = 0.3
communities_selected = []
for community in communities:
    if (len(community) >= size_cap):
        important_nodes = community_central_genes_by_pct(graph,community,pct = pct)
        communities_selected.append(important_nodes)

In [11]:
print(communities_selected)
print(len(communities_selected))

[[7473, 6261, 5260, 12411, 4420, 9023, 602, 1925, 4012, 7756, 7609, 13327, 5665, 3265, 12394, 7552, 8546, 11552, 4781, 3979, 7727, 6847, 12724, 3990, 11858, 4353, 13301, 6814, 5534, 18805, 8659, 7627, 13686, 3753, 8879, 4884, 3432, 10180, 6660, 5333, 5027, 3649, 19516, 13137, 4188, 5509, 3666, 10226, 9597, 12820, 9354, 243, 7219, 10447, 1307, 10804, 2235, 7620, 7249, 2314, 6246, 6981, 12407, 12725, 405, 2874, 5687, 8153, 11047, 13573, 8693, 18139, 5893, 12593, 3771, 6686, 8878, 6324, 6836, 6882, 3148, 6419, 7296, 8880, 12963, 11086, 14841, 10713, 6003, 2635, 12396, 11040, 8414, 5361, 8105, 4310, 6253, 12853, 8617, 8632, 5540, 10698, 10134, 8711, 12304, 12376, 12505, 6698, 3561, 14912, 5073, 6772, 16587, 8412, 6980, 7813, 17199, 4187, 6571, 14932, 9014, 11362, 18708, 7532, 11373, 12306, 6576, 13483, 13609, 16006, 10282, 14386, 7341, 16383, 11237, 10594, 11250, 11794, 3310, 3818, 9705, 13266, 12904, 5187, 16128, 11011, 14533, 16597, 9916, 11411, 3992, 6873, 8922, 6620, 6538, 795, 6762, 1

In [12]:
# Convert index to ncbi
communities_ncbi = [list(map(index_to_gene.get, c)) for c in communities_selected]
print(communities_ncbi)

[['3887', '2098', '677', '10938', '8858', '6167', '9340', '1215', '4926', '4294', '4085', '23581', '1234', '6203', '10916', '4007', '5515', '9871', '14', '7184', '4242', '2998', '11342', '4325', '10235', '5229', '23544', '2959', '1028', '84539', '5648', '4107', '26118', '84870', '5934', '154', '6514', '7789', '2777', '777', '344', '6281', '93058', '23326', '1769', '1001', '3113', '7932', '6892', '22913', '6595', '2328', '3565', '8446', '28', '8896', '6716', '4099', '3605', '1768', '2066', '3170', '10933', '11343', '5747', '2239', '1272', '4993', '9253', '25940', '5693', '80349', '1551', '11159', '7433', '2805', '5933', '2182', '2984', '3040', '2593', '2306', '3667', '5935', '23113', '9311', '51283', '8789', '1730', '6810', '10919', '9244', '5330', '816', '4914', '22916', '2073', '22971', '5595', '5610', '1036', '8771', '7712', '5713', '10799', '10897', '11051', '2822', '130', '51374', '399', '2906', '57171', '5328', '3169', '4437', '64241', '4281', '2645', '51411', '6156', '9656', '842

In [13]:
# NCBI to HGNC symbol
communities_HGNC = []
missed = []


for community in communities_ncbi:
    mg = mygene.MyGeneInfo()
    entrez_ids = [str(e) for e in community]

    results = mg.querymany(
        entrez_ids,
        scopes="entrezgene",
        fields="symbol",
        species="human"
    )

    # Build a mapping: input ID -> symbol (or None)
    id_to_symbol = {}
    for r in results:
        q = str(r.get("query"))
        id_to_symbol[q] = r.get("symbol") if not r.get("notfound") else None

    # Preserve original order
    symbols = [id_to_symbol.get(str(e), None) for e in entrez_ids]
    communities_HGNC.append(symbols)


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
22 input query terms found dup hits:	[('2998', 2), ('2959', 2), ('3113', 2), ('5933', 2), ('2593', 2), ('1805', 2), ('2581', 2), ('3690',
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
12 input query terms found dup hits:	[('9947', 2), ('8638', 2), ('2035', 2), ('51311', 2), ('1889', 2), ('2204', 2), ('1369', 2), ('5395'
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
13 input query terms found dup hits:	[('1376', 2), ('3213', 2), ('8503', 2), ('23461', 2), ('2580', 2), ('597', 2), 

In [14]:
print(communities_HGNC)

[['KRT81', 'ESD', 'ZFP36L1', 'EHD1', 'PROZ', 'RPL37', 'GLP2R', 'CMA1', 'NUMA1', 'MAP3K10', 'MAD2L1', 'CASP14', 'CCR5', 'RPS9', 'MAGED2', 'PRICKLE3', 'PPP2CA', 'SEC24D', 'AAMP', 'HSP90B1', 'MFNG', 'GYS2', 'RNF13', 'MMP16', 'RASGRP2', 'PGGT1B', 'SEZ6L', 'GTF2B', 'CDKN1C', 'MCHR2', 'MASP1', 'MAGEA8', 'WSB1', 'RSPO3', 'RBL2', 'ADRB2', 'SLC2A2', 'ZXDA', 'GNAQP1', 'CACNA1E', 'APOC2', 'S100A10', 'COQ10A', 'USP22', 'DNAH8', 'CDH3', 'HLA-DPA1', 'OR2H2', 'TAPBP', 'RALY', 'SMARCA2', 'FMO3', 'IL4', 'DUSP11', 'ABO', 'BUD31', 'SRD5A2', 'MAG', 'IL17A', 'DNAH6', 'ERBB4', 'FOXA2', 'MORF4L1', 'MGLL', 'PTK2', 'GPC4', 'CNTN1', 'OR2C1', 'NUMBL', 'FAM98A', 'PSMB5', 'SKIC8', 'CYP3A7', 'RABL2A', 'VIPR1', 'GOT1', 'RBL1', 'ACSL4', 'GUCY2C', 'HBA2', 'GAMT', 'FOXD2', 'IRS1', 'RBM3', 'CUL9', 'ASIC3', 'BFAR', 'FBP2', 'DIAPH2', 'STX4', 'EHMT2', 'CRLF1', 'PLCB2', 'CAMK2B', 'NTRK1', 'NCBP2', 'ERCC5', 'RPL41P1', 'MAPK3', 'EIF2AK2', 'CDO1', 'TNFRSF6B', 'ZNF157', 'PSMD7', 'RPP40', 'YIF1A', 'NUDT21', 'GPLD1', 'ADH6', 'ATR

In [15]:
# term_size_cap = 20
term_score_cap = 0.001
percentage = 0.3
important_terms = pd.DataFrame(columns=["Community Index","Community Size","Term", "Overlap", "Adjusted P-value"])

In [16]:
# GO Analysis; save terms with small size and high p-value
i = 0
for community in communities_HGNC:
    # Gene Ontology enrichment
    enr_go = gp.enrichr(
        gene_list=community,
        gene_sets=['GO_Biological_Process_2023',
                'GO_Molecular_Function_2023',
                'GO_Cellular_Component_2023'],
        organism='Human',
        outdir=None # don't write to disk
    )
    go_df = enr_go.results
    go_df = go_df.sort_values('Adjusted P-value')
    print(f"Size of community: {len(community)}")

    mask =  (go_df["Adjusted P-value"] < term_score_cap) & (go_df["Overlap"].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]) > percentage))
    filtered = go_df[mask].copy()
    if not filtered.empty:
        filtered.loc[:, "Community Index"] = i
        filtered.loc[:, "Community Size"] = len(community)
        important_terms = pd.concat([important_terms, filtered], ignore_index=True)

    display(HTML(go_df[['Term','Overlap','Adjusted P-value']].head(10).to_html(max_cols=None)))
    i += 1



Size of community: 1193


C:\Users\celem\AppData\Local\Temp\ipykernel_32720\2327067429.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  important_terms = pd.concat([important_terms, filtered], ignore_index=True)


,Term,Overlap,Adjusted P-value
0,Regulation Of Cell Population Proliferation (GO:0042127),83/766,0.000154
3873,Zinc Ion Binding (GO:0008270),45/341,0.000220
2,Positive Regulation Of Cellular Process (GO:0048522),66/594,0.000500
3,Receptor Signaling Pathway Via JAK-STAT (GO:0007259),12/36,0.000500
1,Positive Regulation Of Cell Population Proliferation (GO:0008284),57/483,0.000500
3874,Transition Metal Ion Binding (GO:0046914),53/456,0.000578
5,Positive Regulation Of Cell Motility (GO:2000147),32/221,0.001289
4,Positive Regulation Of Protein Phosphorylation (GO:0001934),46/377,0.001289
6,Positive Regulation Of Cell Migration (GO:0030335),36/272,0.002218
7,Receptor Signaling Pathway Via STAT (GO:0097696),10/31,0.002899


Size of community: 957


,Term,Overlap,Adjusted P-value
0,Positive Regulation Of Cytokine Production (GO:0001819),40/320,0.000074
3635,DNA Binding (GO:0003677),73/846,0.000351
1,Regulation Of DNA-templated Transcription (GO:0006355),135/1922,0.002708
4381,Intracellular Membrane-Bounded Organelle (GO:0043231),301/5175,0.003320
4382,Ficolin-1-Rich Granule (GO:0101002),23/184,0.003320
2,Positive Regulation Of DNA-templated Transcription (GO:0045893),94/1243,0.004030
3,Positive Regulation Of Macromolecule Biosynthetic Process (GO:0010557),19/122,0.004030
4383,Nucleus (GO:0005634),263/4487,0.004408
4384,Intracellular Organelle Lumen (GO:0070013),66/856,0.005145
6,Protein Phosphorylation (GO:0006468),46/500,0.006768


...

,Term,Overlap,Adjusted P-value
169,Diphosphotransferase Activity (GO:0016778),1/5,0.111222
171,BMP Receptor Binding (GO:0070700),1/10,0.111222
172,Transmembrane Receptor Protein Serine/Threonine Kinase Binding (GO:0070696),1/10,0.111222
173,RNA Polymerase I Activity (GO:0001054),1/11,0.111222
174,Sequence-Specific mRNA Binding (GO:1990825),1/11,0.111222
170,pre-mRNA Intronic Binding (GO:0097157),1/8,0.111222
175,Acylglycerol Lipase Activity (GO:0047372),1/14,0.121016
178,mRNA 3'-UTR AU-rich Region Binding (GO:0035925),1/23,0.138081
176,Triglyceride Lipase Activity (GO:0004806),1/21,0.138081
177,Sequence-Specific DNA Binding (GO:0043565),4/717,0.138081


Size of community: 26


,Term,Overlap,Adjusted P-value
295,Axon (GO:0030424),3/205,0.030724
294,Synaptic Vesicle Membrane (GO:0030672),2/54,0.030724
293,Exocytic Vesicle Membrane (GO:0099501),2/53,0.030724
292,Phagocytic Vesicle Membrane (GO:0030670),2/45,0.030724
0,Negative Regulation Of Viral-Induced Cytoplasmic Pattern Recognition Receptor Signaling Pathway (GO:0039532),2/16,0.046665
296,Neuron Projection (GO:0043005),4/557,0.057948
297,Phagocytic Vesicle (GO:0045335),2/100,0.065711
298,Spectrin-Associated Cytoskeleton (GO:0014731),1/7,0.068641
8,Axon Development (GO:0061564),2/99,0.091701
30,Regulation Of Axon Regeneration (GO:0048679),1/10,0.091701


Size of community: 15


,Term,Overlap,Adjusted P-value
175,4-galactosyl-N-acetylglucosaminide 3-alpha-L-fucosyltransferase Activity (GO:0017083),1/5,0.028744
176,Serine-Type Endopeptidase Activity (GO:0004252),2/125,0.028744
185,Metalloendopeptidase Inhibitor Activity (GO:0008191),1/13,0.028744
184,GDP-dissociation Inhibitor Activity (GO:0005092),1/12,0.028744
183,P-type Ion Transporter Activity (GO:0015662),1/11,0.028744
182,Fucosyltransferase Activity (GO:0008417),1/11,0.028744
186,Solute:Inorganic Anion Antiporter Activity (GO:0005452),1/14,0.028744
180,P-type Calcium Transporter Activity (GO:0005388),1/8,0.028744
179,Alpha-(1->3)-Fucosyltransferase Activity (GO:0046920),1/7,0.028744
178,Natural Killer Cell Lectin-Like Receptor Binding (GO:0046703),1/7,0.028744


In [ ]:
# KEGG and Reactome enrichment
i = 0
for community in communities_HGNC:
    enr_path = gp.enrichr(
        gene_list=community,
        gene_sets=['KEGG_2021_Human','Reactome_2022'],
        organism='Human',
        outdir=None
    )
    enr_path_df = enr_path.results
    enr_path_df = enr_path_df.sort_values('Adjusted P-value')

    mask = (enr_path_df["Overlap"].apply(lambda x: int(x.split("/")[1]) <= term_size_cap)) & (enr_path_df["Adjusted P-value"] <= term_score_cap)
        
    filtered = enr_path_df[mask].copy()
    if not filtered.empty:
        filtered.loc[:, "Community Index"] = i
        filtered.loc[:, "Community Size"] = len(community)
        important_terms = pd.concat([important_terms, filtered], ignore_index=True)

    print(f"Size of community: {len(community)}")
    display(HTML(enr_path_df[['Term','Overlap','Adjusted P-value']].head(10).to_html(max_cols=None)))
    i += 1

Size of community: 1334


,Term,Overlap,Adjusted P-value
204,Protein Methylation R-HSA-8876725,7/17,0.043365
205,Detoxification Of Reactive Oxygen Species R-HSA-3299685,8/34,0.440789
206,Serine Biosynthesis R-HSA-977347,4/9,0.440789
207,Miscellaneous Transport And Binding Events R-HSA-5223345,6/23,0.573481
208,eNOS Activation R-HSA-203615,4/11,0.592016
209,RUNX3 Regulates Immune Response And Cell Migration R-HSA-8949275,3/6,0.592016
0,Peroxisome,13/82,0.595353
211,TP53 Regulates Transcription Of Cell Death Genes R-HSA-5633008,8/44,0.678663
212,TP53 Regulates Transcription Of Genes Involved In Cytochrome C Release R-HSA-6803204,5/20,0.678663
210,Smooth Muscle Contraction R-HSA-445355,8/43,0.678663


Size of community: 1188


,Term,Overlap,Adjusted P-value
211,Signal Transduction R-HSA-162582,282/2465,1.932896e-26
212,Immune System R-HSA-168256,230/1943,6.003656e-23
213,"Signaling By Rho GTPases, Miro GTPases And RHOBTB3 R-HSA-9716542",109/660,2.752168e-20
214,RHO GTPase Cycle R-HSA-9012999,85/441,4.933620e-20
215,Signaling By Rho GTPases R-HSA-194315,106/644,8.748492e-20
0,MAPK signaling pathway,63/294,5.332700e-17
216,Signaling By Receptor Tyrosine Kinases R-HSA-9006934,84/496,2.976105e-16
217,Adaptive Immune System R-HSA-1280218,99/733,1.392912e-12
218,Class I MHC Mediated Antigen Processing And Presentation R-HSA-983169,64/378,2.954294e-12
219,Antigen Processing: Ubiquitination And Proteasome Degradation R-HSA-983168,56/307,4.531411e-12


Size of community: 985


...

Size of community: 37


c:\Users\celem\Research\DGI-Hypergraph-Fin\.venv\lib\site-packages\gseapy\enrichr.py:689: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat(self.results, ignore_index=True)


,Term,Overlap,Adjusted P-value
0,RNA transport,1/186,0.493494
1,Amyotrophic lateral sclerosis,1/364,0.493494


Size of community: 25


,Term,Overlap,Adjusted P-value
111,Notch-HLH Transcription Pathway R-HSA-350054,10/28,2.810563e-21
112,NOTCH1 Intracellular Domain Regulates Transcription R-HSA-2122947,10/48,6.911065e-19
113,Constitutive Signaling By NOTCH1 HD+PEST Domain Mutants R-HSA-2894862,10/58,3.650886e-18
114,Signaling By NOTCH1 R-HSA-1980143,10/74,3.729275e-17
0,Alcoholism,11/186,1.474819e-14
1,Neutrophil extracellular trap formation,10/189,7.167032e-13
2,Viral carcinogenesis,10/203,9.835364e-13
115,Signaling By NOTCH R-HSA-157118,10/203,9.941692e-13
116,Diseases Of Signal Transduction By Growth Factor Receptors And Second Messengers R-HSA-5663202,11/424,3.635017e-11
117,HDACs Deacetylate Histones R-HSA-3214815,5/60,2.645048e-07


In [26]:
important_terms = important_terms.sort_values(by="Overlap")
display(HTML(important_terms[["Community Index","Community Size",'Term','Overlap','Adjusted P-value',"P-value"]].head(50).to_html(max_cols=None)))

,Community Index,Community Size,Term,Overlap,Adjusted P-value,P-value
13,1,1188,Regulation Of Cell-Substrate Junction Assembly (GO:0090109),11/21,1.925791e-07,6.301672e-09
22,1,1188,Positive Regulation Of p38MAPK Cascade (GO:1900745),11/24,9.193603e-07,3.778523e-08
21,1,1188,Peptidyl-Tyrosine Dephosphorylation (GO:0035335),12/29,9.025130e-07,3.685655e-08
26,6,555,Postsynaptic Specialization Membrane (GO:0099634),12/38,3.663528e-08,2.598247e-10
18,1,1188,Negative Regulation Of Mitotic Cell Cycle (GO:0045930),13/34,7.908138e-07,3.084588e-08
23,5,652,Phosphotyrosine Residue Binding (GO:0001784),13/39,4.533111e-08,1.552435e-10
25,6,555,Neuropeptide Receptor Activity (GO:0008188),14/36,9.312310e-12,2.956289e-13
20,1,1188,Regulation Of Microtubule Polymerization (GO:0031113),14/40,8.450087e-07,3.406580e-08
14,1,1188,Negative Regulation Of I-kappaB kinase/NF-kappaB Signaling (GO:0043124),15/42,2.385459e-07,7.993160e-09
19,1,1188,Regulation Of Epidermal Growth Factor Receptor Signaling Pathway (GO:0042058),15/46,8.318890e-07,3.310134e-08


In [18]:
disease_size_cap = 200
disease_score_cap = 0.0001
important_diseases = pd.DataFrame(columns=["Community Index","Community Size","Term", "Overlap", "Adjusted P-value"])

In [19]:
# Disease-gene enrichment libraries
disease_sets = [
    'DisGeNET_2020', # curated gene–disease associations
    'GWAS_Catalog_2023', # genome-wide association hits
    'OMIM_Disease', # Mendelian disorders
    'Jensen_DISEASES' # text-mined associations
]

# # Disease-gene enrichment Analysis; save terms with small size and high p-value
i = 0
for community in communities_HGNC:
    # Gene Ontology enrichment
    enr_disease = gp.enrichr(
        gene_list=community,
        gene_sets=disease_sets,
        organism='Human',
        outdir=None # don't write to disk
    )
    enr_disease_df = enr_disease.results.sort_values('Adjusted P-value')
    print(f"Size of community: {len(community)}")

    mask = ((enr_disease_df["Overlap"].apply(lambda x: int(x.split("/")[1]) < disease_size_cap)) 
    & (enr_disease_df["Adjusted P-value"] < disease_score_cap))
        
    filtered = enr_disease_df[mask].copy()
    if not filtered.empty:
        filtered.loc[:, "Community Index"] = i
        filtered.loc[:, "Community Size"] = len(community)
        important_diseases = pd.concat([important_diseases, filtered], ignore_index=True)

    display(HTML(enr_disease_df[['Term','Overlap','Adjusted P-value']].head(10).to_html(max_cols=None)))
    i += 1

2025-10-22 13:04:59,519 [WARNING] Input library not found: DisGeNET_2020. Skip


Size of community: 1334


,Term,Overlap,Adjusted P-value
3042,Peroxisomal disease,6/14,0.092750
3043,Vesicoureteral reflux,8/26,0.092750
3044,Amelogenesis imperfecta,6/18,0.245657
0,Vaginal Microbiome MetaCyc Pathway (PWY-1622|formaldehyde Assimilation I (Serine Pathway)),8/24,0.322647
3045,Hermansky-Pudlak syndrome,4/9,0.341460
3046,Aicardi syndrome,5/15,0.341460
3047,Mitral valve prolapse,5/15,0.341460
3048,Intermediate coronary syndrome,3/5,0.349828
1,2-Hydroxy-3-Methylvalerate Levels In Elite Athletes,4/6,0.397831
3049,Zellweger syndrome,4/11,0.494349


2025-10-22 13:05:09,294 [WARNING] Input library not found: DisGeNET_2020. Skip


Size of community: 1188


...

c:\Users\celem\Research\DGI-Hypergraph-Fin\.venv\lib\site-packages\gseapy\enrichr.py:689: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat(self.results, ignore_index=True)


,Term,Overlap,Adjusted P-value
0,Lymphoma,1/5,0.153869
1,Asthma And Major Depressive Disorder,1/5,0.153869
2,F-savour/caloric Food Liking (Derived Food-Liking Factor),1/7,0.153869
3,F-highly Palatable Foods Liking (Derived Food-Liking Factor),1/10,0.153869
4,F-cake/biscuits Liking (Derived Food-Liking Factor),1/10,0.153869
5,Angioedema In Response To Angiotensin-Converting Enzyme Inhibitor And/Or Angiotensin Receptor Blocker,1/10,0.153869
6,Suicidal Thoughts And Behaviors,1/11,0.153869
7,Gait Speed In Old Age,1/12,0.153869
8,Caffeine Consumption From Tea,1/18,0.204058
9,Gut Microbiome Composition (Summer),1/22,0.223661


2025-10-22 13:06:30,344 [WARNING] Input library not found: DisGeNET_2020. Skip


Size of community: 25


,Term,Overlap,Adjusted P-value
402,Succinic semialdehyde dehydrogenase deficiency,2/16,0.016315
403,Endometrial cancer,4/292,0.016315
404,Large intestine cancer,3/122,0.016315
405,DOID:9917,2/30,0.016925
392,ehlers-danlos,1/11,0.030989
394,long qt syndrome,1/12,0.030989
395,melanoma,1/13,0.030989
396,cone-rod dystrophy,1/15,0.030989
397,hypogonadism,1/15,0.030989
393,osteoporosis,1/11,0.030989


In [20]:
important_diseases = important_diseases.sort_values(by="Community Index")
display(HTML(important_diseases[["Community Index","Community Size",'Term','Overlap','Adjusted P-value']].head(50).to_html(max_cols=None)))

,Community Index,Community Size,Term,Overlap,Adjusted P-value
0,5,652,Vertical Cup-Disc Ratio,23/193,0.000035
1,5,652,Vertical Cup-Disc Ratio (Adjusted For Vertical Disc Diameter),12/55,0.000045
2,5,652,Vertical Cup-Disc Ratio (Multi-Trait Analysis),12/58,0.000061
3,5,652,Respiratory Diseases,14/80,0.000061
4,5,652,Glaucoma (Primary Open-Angle),15/92,0.000061
5,11,74,Hemolytic anemia,5/72,0.000084
